In [5]:
import os
from openai import OpenAI

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
    model='gpt-4o-mini-2024-07-18',
    messages=[{'role':'user', 'content':'gpt-4o와 gpt-4o-mini-2024-07-18의 차이에 대해 알려줘'}],
    temperature=0.0
)

print(completion.choices[0].message.content)


GPT-4o와 GPT-4o-mini-2024-07-18은 OpenAI의 두 가지 모델로, 주로 크기와 성능, 용도에 따라 차이가 있습니다. 

1. **모델 크기**:
   - **GPT-4o**: 일반적으로 더 큰 모델로, 더 많은 파라미터를 가지고 있어 복잡한 작업을 수행하는 데 유리합니다.
   - **GPT-4o-mini**: 상대적으로 작은 모델로, 경량화되어 있어 빠른 응답 속도와 낮은 자원 소모가 특징입니다.

2. **성능**:
   - **GPT-4o**: 더 높은 성능을 제공하며, 복잡한 질문이나 작업에 대해 더 깊이 있는 답변을 생성할 수 있습니다.
   - **GPT-4o-mini**: 성능은 다소 떨어질 수 있지만, 여전히 많은 작업을 수행할 수 있으며, 특히 자원이 제한된 환경에서 유용합니다.

3. **용도**:
   - **GPT-4o**: 연구, 고급 애플리케이션, 복잡한 대화 시스템 등에서 사용됩니다.
   - **GPT-4o-mini**: 모바일 애플리케이션, 실시간 응답이 필요한 서비스 등에서 더 적합합니다.

4. **업데이트 날짜**:
   - **GPT-4o-mini-2024-07-18**: 이 모델은 특정 날짜에 업데이트된 버전으로, 최신 데이터와 개선 사항이 반영되어 있을 가능성이 높습니다.

이러한 차이점들은 사용자의 필요에 따라 적합한 모델을 선택하는 데 도움이 됩니다.


In [8]:
import json

with open('./resource/reviews.json', 'r') as f:
    reviews = json.load(f)

reviews[:3]

[{'review': '적절한 가격에 9시간 스테이를 활용할 수 있어서 좋았습니다.', 'stars': 5, 'date': '1일 전'},
 {'review': '시간대 선택이 좋으네요', 'stars': 4, 'date': '1일 전'},
 {'review': '지하철 역과 가깝고 코엑스 및 주변 부대시설과 접근이 용이해요~',
  'stars': 5,
  'date': '2024.09.04'}]

In [17]:
# good = 별점 5개
# bad = 별점 4개 이하
SPECIAL_TOKEN_PREFIX = '[REVIEW_START]'
SPECIAL_TOKEN_SUFFIX = '[REVIEW_END]'
good_reviews, bad_reviews = [], []
for r in reviews:
    if r['stars'] >= 5:
        good_reviews.append(f"{SPECIAL_TOKEN_PREFIX}{r['review']}{SPECIAL_TOKEN_SUFFIX}")
    else:
        bad_reviews.append(f"{SPECIAL_TOKEN_PREFIX}{r['review']}{SPECIAL_TOKEN_SUFFIX}")

print(good_reviews[:3])
print(bad_reviews[:3])
print(len(good_reviews), len(bad_reviews))

good_reviews_text = '\n'.join(good_reviews)
bad_reviews_text = '\n'.join(bad_reviews)

bad_reviews_text[:100]

['[REVIEW_START]적절한 가격에 9시간 스테이를 활용할 수 있어서 좋았습니다.[REVIEW_END]', '[REVIEW_START]지하철 역과 가깝고 코엑스 및 주변 부대시설과 접근이 용이해요~[REVIEW_END]', '[REVIEW_START]깨끗하고 부대시설이 좋았어요!![REVIEW_END]']
['[REVIEW_START]시간대 선택이 좋으네요[REVIEW_END]', '[REVIEW_START]진짜 작아요 정말로 그냥접근성 하나만보고 잤어요[REVIEW_END]', '[REVIEW_START]네 그냥 전반적으로 그냥 그래요[REVIEW_END]']
165 55


'[REVIEW_START]시간대 선택이 좋으네요[REVIEW_END]\n[REVIEW_START]진짜 작아요 정말로 그냥접근성 하나만보고 잤어요[REVIEW_END]\n[REVIEW_'

In [19]:
import datetime
from dateutil import parser

# 전처리 함수
def preprocess_reviews(path='./resource/reviews.json'):
    
    with open(path, 'r', encoding='utf-8') as f:
        reviews = json.load(f)

    # good = 별점 5개
    # bad = 별점 4개 이하
    SPECIAL_TOKEN_PREFIX = '[REVIEW_START]'
    SPECIAL_TOKEN_SUFFIX = '[REVIEW_END]'
    good_reviews, bad_reviews = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    for r in reviews:
        review_date_string = r['date']
        try:
            review_date = parser.parse(review_date_string)
        except (ValueError, TypeError):
            # '22시간 전' 같은 문자로 된 날짜는 현재 날짜로 처리
            review_date = current_date

        # 6개월 이전 리뷰는 제외
        if review_date < date_boundary:
            continue

        if r['stars'] >= 5:
            good_reviews.append(f"{SPECIAL_TOKEN_PREFIX}{r['review']}{SPECIAL_TOKEN_SUFFIX}")
        else:
            bad_reviews.append(f"{SPECIAL_TOKEN_PREFIX}{r['review']}{SPECIAL_TOKEN_SUFFIX}")

    good_reviews_text = '\n'.join(good_reviews)
    bad_reviews_text = '\n'.join(bad_reviews)

    return good_reviews_text, bad_reviews_text

good, bad = preprocess_reviews()


'[REVIEW_START]적절한 가격에 9시간 스테이를 활용할 수 있어서 좋았습니다.[REVIEW_END]\n[REVIEW_START]지하철 역과 가깝고 코엑스 및 주변 부대시설과 '

평가용 함수 작성
평가 기준 설정

MT-Bench 논문 기반 Pairwise Comparision (=LLM 기반 평가)
비교하는 방식 vs. 점수 매기는 방식
점수라는게 애매 할 수 있음 (ex. 어느 정도의 요약 품질이 3점인가?)
경험상 점수보다는 비교가 상대적으로 더 정확한 편
평가 스크립트 작성

MT-Bench 논문 Prompt에서 일부 단어만 수정 (ex. Korean summaries, accommodations)

In [22]:
def pairwise_eval(reviews, answer_a, answer_b):
    eval_prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the Korean summaries provided by two
AI assistants to the set of user reviews on accommodations displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.
[User Reviews]
{reviews}
[The Start of Assistant A’s Answer]
{answer_a}
[The End of Assistant A’s Answer]
[The Start of Assistant B’s Answer]
{answer_b}
[The End of Assistant B’s Answer]"""
    
    completion = client.chat.completions.create(
        model='gpt-4o-mini-2024-07-18',
        messages=[{'role': 'user', 'content': eval_prompt}],
        temperature=0.0
    )

    return completion.choices[0].message.content

# Baseline 모델 개발

In [20]:
PROMPT_BASELINE = f"""아래 숙소 리뷰에 대해 5문장 내로 요약해줘:"""

reviews, _ = preprocess_reviews(path = './resource/reviews.json')

def summarize(reviews, prompt, temperature=0.0, model='gpt-4o-mini-2024-07-18'):
    prompt = prompt + '\n\n' + reviews

    completion = client.chat.completions.create(
        model=model,
        messages=[{'role':'user', 'content':prompt}],
        temperature=temperature
    )

    return completion.choices[0].message.content

print(summarize(reviews, PROMPT_BASELINE))

이 숙소는 삼성역 근처에 위치해 있어 접근성이 뛰어나며, 코엑스와의 근접성도 장점으로 꼽힙니다. 객실은 깨끗하고 편안하며, 친절한 직원들이 인상적입니다. 가격 대비 가성비가 좋고, 셀프 체크인 시스템이 편리하다는 리뷰도 많습니다. 조식은 아메리칸 스타일로 제공되어 만족스럽고, 전반적으로 쾌적한 분위기를 제공합니다. 재방문 의사가 높은 고객들이 많아 긍정적인 평가가 이어지고 있습니다.


In [26]:
summary_real_20240915 = '삼성역과 가까운 위치로 접근성이 뛰어나며, 코엑스와의 근접성도 장점으로 언급되었습니다. 객실의 청결 상태와 친절한 직원 서비스가 높은 평가를 받고 있으며, 조식과 부대시설 또한 만족스러운 것으로 확인되었습니다. 다만, 도로 근처에 위치해 소음이 발생할 수 있으며, 일부 청소 상태에서 아쉬운 점이 지적되었습니다. 가격 대비 가성비가 좋으며, 전반적으로 편안한 숙박 경험을 제공하는 시설입니다.'
bad_summary_real_20240915 = '삼성역 1번출구와 가까운 위치에 있어 접근성이 뛰어납니다. 깨끗하고 청결한 편이나 객실이 상당히 작은 점이 아쉽다는 의견이 있습니다. 넷플릭스 이용이 가능하며, 조식은 맛있다는 평가도 있습니다. 고층으로 배정받은 경우 소음이 덜하였고, 그러나 주변 공사로 인한 소음 문제가 지적되기도 했습니다. 시설 및 서비스는 보통이며 예약이 편리하다는 장점이 있습니다.'


In [25]:
print(pairwise_eval(reviews, summarize(reviews, PROMPT_BASELINE), summary_real_20240915))

Both assistants provided summaries of the user reviews, but there are notable differences in their responses.

**Assistant A:**
- Focuses on the positive aspects of the accommodation, such as its proximity to Samsung Station and COEX, the cleanliness and comfort of the rooms, the friendly staff, and the convenience of the self-check-in system.
- Mentions the American-style breakfast and the overall good value for money.
- Briefly notes that parking is paid but does not delve into any negative aspects.

**Assistant B:**
- Also highlights the positive aspects like the location near Samsung Station and COEX, the cleanliness of the rooms, and the friendly staff.
- Adds details about the breakfast and facilities being satisfactory.
- Includes potential downsides such as noise from the nearby road and some issues with the cleaning state.
- Emphasizes the good value for money and overall comfortable stay.

**Evaluation:**
- **Helpfulness:** Both summaries are helpful, but Assistant B provides

### 대규모 평가 스크립트
- 원래는 많은 수의 다양한 데이터에 대해 평가를 하지만, 동일한 Prompt에 대해 temperature 높여서 평가 진행

In [27]:
eval_count = 10

summaries_baseline = [summarize(reviews, PROMPT_BASELINE, temperature=1.0) for _ in range(eval_count)]
summaries_baseline

['이 숙소는 가격이 합리적이며 삼성역과 가까워 접근성이 뛰어나고 위치가 좋습니다. 객실은 깔끔하고 청결하며 직원들도 친절하여 이용하기 편리합니다. 조식이 포함되어 있어 알찬 선택이 가능하고, 셀프 체크인 시스템으로 편안하게 이용할 수 있습니다. 다양한 부대시설이 갖춰져 있어 출장이나 여행 중 편안한 휴식을 제공합니다. 전반적으로 매우 만족스러운 경험을 제공하는 호텔입니다.',
 '이 숙소는 우수한 가격과 쾌적한 환경 덕분에 많은 이용자들에게 호평받고 있습니다. 위치가 삼성역과 가까워 접근성이 뛰어나며, 코엑스와의 근접성도 큰 장점으로 꼽힙니다. 숙소는 매우 깨끗하고 친절한 직원들이 좋은 서비스를 제공하며, 편안한 침구와 조식도 만족스럽다는 리뷰가 많습니다. 고객들은 셀프 체크인 시스템과 주차 편의성도 긍정적으로 평가하고 있습니다. 전반적으로 재방문 의사를 밝히는 리뷰가 많아 전반적인 만족도가 높은 곳입니다.',
 '리뷰 요약: 이 숙소는 접근성이 뛰어나고, 깨끗하며, 친절한 스태프가 인상적이라는 평가가 많습니다. 삼성역과 가까워 편리하며, 조식도 만족스럽고 편안한 침구로 휴식을 취하기 좋습니다. 가격 대비 훌륭한 가성비가 밝혀졌고, 다양한 편의시설이 잘 갖춰져 있습니다. 전체적으로 만족스러운 경험을 제공하는 곳으로, 재방문 의사가 높습니다.',
 '이 숙소는 삼성역에 가까워 접근성이 뛰어나며, 깨끗하고 편안한 환경을 제공합니다. 직원들이 친절하고, 셀프 체크인 시스템이 편리해 많은 방문객들이 만족하고 있습니다. 조식은 훌륭하며, 다양한 편의시설도 가까이에 있어 편리한 숙박을 제공합니다. 여러 리뷰에서 가성비와 청결, 친절한 서비스가 특히 언급되고 있습니다. 전반적으로 방문객들은 이곳에서 편안하고 즐거운 시간을 보냈다는 긍정적인 평가를 남겼습니다.',
 '전체적으로 리뷰에서 언급된 바에 따르면, 이 숙소는 삼성역 바로 앞에 위치해 접근성이 매우 좋으며, 주변에 다양한 편의시설이 있다는 점에서 높은 평가를 받고 있습니다. 객실은 깨끗하고 쾌적하며, 침구 상태도 

In [28]:
from tqdm import tqdm

def pairwise_eval_batch(reviews, answers_a, answers_b):
    a_count, b_count, draw_count = 0, 0, 0
    for i in tqdm(range(len(answers_a))):
        result = pairwise_eval(reviews, answers_a[i], answers_b[i])
        verdict_text = result

        if '[[A]]' in verdict_text:
            a_count += 1
        elif '[[B]]' in verdict_text:
            b_count += 1
        elif '[[C]]' in verdict_text:
            draw_count += 1
        else:
            print('Evaluation Error')
    return a_count, b_count, draw_count

wins, losses, ties = pairwise_eval_batch(reviews, summaries_baseline, [summary_real_20240915 for _ in range(len(summaries_baseline))])

print(f'Wins : {wins}, Losses : {losses}, Ties : {ties}')


100%|██████████| 10/10 [00:40<00:00,  4.04s/it]

Wins : 3, Losses : 7, Ties : 0


### 모델 고도화1 - 조건들 명시

In [30]:
prompt = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0) for _ in range(eval_count)]
summaries

['이번 숙소 리뷰에서는 접근성과 청결, 친절한 서비스에 대한 긍정적인 피드백이 두드러지게 나타났습니다. 삼성역과 코엑스몰 근처에 위치하여 대중교통 이용이 용이하고, 객실이 깔끔하다는 점이 호평받고 있습니다. 또한, 스태프의 친절함과 편리한 체크인 시스템 덕분에 편안한 숙박 경험을 하셨다고 말씀하셨습니다. 전반적으로 매우 만족스러운 숙소인 것 같습니다.',
 '글래드 코엑스점은 우수한 위치와 접근성으로 많은 분들께 사랑받는 숙소입니다. 깨끗한 객실과 친절한 직원들 덕분에 편안한 숙박 경험을 제공하며, 특히 무인 셀프 체크인 시스템이 편리하다는 평가가 많습니다. 숙소에서 제공하는 조식도 아침 식사로 적절하며, 다양한 편의시설이 근처에 위치해 있어 더욱 만족스러운 선택이 될 것 같습니다. 재방문 의사가 높은 리뷰들이 많아, 안심하고 이용하실 수 있는 숙소입니다.',
 '숙소에 대한 리뷰를 종합해보면, 위치와 접근성이 매우 우수하여 삼성역과 코엑스 근처에 있어 편리하다는 평이 많습니다. 많은 이용자분들이 숙소의 청결 상태와 친절한 직원 서비스에 만족하고 있으며, 특히 침구가 편안하다는 점이 자주 언급되고 있습니다. 또한 조식 또한 맛있고 다양하다는 인상이 있습니다. 여러 리뷰에서 재방문 의사가 확인되며, 고객분들이 이 숙소에서 쾌적한 시간을 보내셨음을 잘 알 수 있습니다.',
 '이 숙소는 접근성이 뛰어난 위치에 있으며, 청결하고 아늑한 분위기로 고객들에게 높은 만족도를 주고 있습니다. 삼성역과 가까워 대중교통 이용이 편리하며, 코엑스와의 인접성으로 다채로운 부대시설도 쉽게 이용할 수 있습니다. 직원들의 친절함과 훌륭한 서비스도 많은 호평을 받고 있으며, 조식 또한 맛있다는 의견이 많습니다. 여러 리뷰에서 재방문 의사를 밝히며, 다양한 편의 시설과 쾌적한 숙소 조건으로 좋은 힐링 공간으로 자리 잡고 있다는 평이 있습니다.',
 '글래드 호텔은 적절한 가격에 쾌적한 숙소를 제공하여 많은 손님들에게 사랑받고 있습니다. 삼성역과 가까운 위치 덕분에 코엑스와 다양한 편의시

In [32]:
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240915 for _ in range(eval_count)])

print(f'Wins : {wins}, Losses : {losses}, Ties : {ties}')


100%|██████████| 10/10 [00:50<00:00,  5.01s/it]

Wins : 2, Losses : 8, Ties : 0


### 모델 고도화2 - 입력 데이터의 품질 증가

In [35]:
import datetime
from dateutil import parser

# 전처리 함수 v2 : 내용이 적은 리뷰는 걸러냄
def preprocess_reviews_v2(path='./resource/reviews.json'):
    
    with open(path, 'r', encoding='utf-8') as f:
        reviews = json.load(f)

    # good = 별점 5개
    # bad = 별점 4개 이하
    SPECIAL_TOKEN_PREFIX = '[REVIEW_START]'
    SPECIAL_TOKEN_SUFFIX = '[REVIEW_END]'
    good_reviews, bad_reviews = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    filtered_count = 0
    for r in reviews:
        review_date_string = r['date']
        try:
            review_date = parser.parse(review_date_string)
        except (ValueError, TypeError):
            # '22시간 전' 같은 문자로 된 날짜는 현재 날짜로 처리
            review_date = current_date

        # 6개월 이전 리뷰는 제외
        if review_date < date_boundary:
            continue
        if len(r['review']) < 30:
            filtered_count += 1
            # print(r['review'])
            continue

        if r['stars'] >= 5:
            good_reviews.append(f"{SPECIAL_TOKEN_PREFIX}{r['review']}{SPECIAL_TOKEN_SUFFIX}")
        else:
            bad_reviews.append(f"{SPECIAL_TOKEN_PREFIX}{r['review']}{SPECIAL_TOKEN_SUFFIX}")

    good_reviews_text = '\n'.join(good_reviews)
    bad_reviews_text = '\n'.join(bad_reviews)

    print(f'# filtered_count : {filtered_count}')

    return good_reviews_text, bad_reviews_text

reviews, _ = preprocess_reviews_v2()


# filtered_count : 119


In [37]:
eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0) for _ in range(eval_count)]
summaries

['글래드 코엑스점은 접근성이 뛰어난 위치와 쾌적한 객실로 많은 호평을 받고 있습니다. 고객들은 친절한 스태프와 깨끗한 침구, 만족스러운 조식을 경험하며 재방문 의사를 밝히고 있습니다. 또한, 셀프 체크인 시스템과 편리한 대중교통 이용이 큰 장점으로 언급되고 있습니다. 특히, 다양한 프로모션이 제공되어 더 많은 고객들에게 만족감을 선사하고 있습니다. 전체적으로 편안하고 아늑한 분위기로, 힐링을 원하는 분들에게 적합한 숙소입니다.',
 '글래드 코엑스점은 삼성역과 가까워 교통이 매우 편리하다는 평가를 받고 있습니다. 스태프분들이 친절하고 숙소 내 청결 상태가 우수하여 편안한 숙박을 제공합니다. 또한, 조식은 아메리칸 스타일로 구성되어 만족스럽게 이용할 수 있는 점이 좋았다는 의견도 있었습니다. 재방문 계획을 세우는 많은 분들이 계시며, 전반적으로 쾌적한 환경과 서비스에 대한 긍정적인 인상이 많습니다.',
 '글래드 코엑스점은 뛰어난 위치와 쾌적한 환경으로 많은 호평을 받고 있습니다. 삼성역과 가까워 대중교통 이용이 편리하며, 코엑스 및 여러 부대시설에도 쉽게 접근할 수 있습니다. 스태프들의 친절한 서비스와 청결한 객실, 아침 식사의 만족도 또한 높습니다. 특히 셀프 체크인 시스템이 도입되어 있어 편리함까지 더했습니다. 재방문 의사를 밝히는 고객이 많아 추천할 만한 숙소입니다.',
 '글래드 코엑스점은 삼성역과 가까운 뛰어난 위치 덕분에 교통이 매우 편리하다는 평이 많습니다. 스태프들은 친절하며 청결한 객실과 아침식사가 만족스럽다는 의견이 많이 있습니다. 또한, 셀프 체크인 시스템이 잘 되어 있어 편안하게 이용할 수 있는 점이 장점으로 꼽힙니다. 재방문 의사를 나타낸 글들도 많아, 고객들의 높은 만족도를 알 수 있었습니다. 전반적으로, 편리한 위치와 쾌적한 환경을 갖춘 좋은 숙소로 추천드립니다.',
 '글래드 코엑스점은 적절한 가격에 위치와 접근성이 뛰어난 숙소로 평가받고 있습니다. 삼성역과 가까워 대중교통 이용이 편리하며, 코엑스와 주변 편의시설도 쉽게 이용할 

In [38]:
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240915 for _ in range(eval_count)])

print(f'Wins : {wins}, Losses : {losses}, Ties : {ties}')


100%|██████████| 10/10 [00:40<00:00,  4.07s/it]

Wins : 3, Losses : 7, Ties : 0


### 모델 고도화3 - Few-Shot Prompting

In [39]:
reviews_1shot, _ = preprocess_reviews_v2('./resource/novotel_reviews.json')
# summary_1shot = summarize(reviews_1shot, prompt, temperature=0.0)
summary_1shot_real = """용산역과 가까워 접근성이 뛰어나며, 근처에 아이파크몰이 위치해 있어 편리합니다. 객실은 넓고 청결하며, 편안한 침대와 훌륭한 도심 뷰가 특징입니다. 다양한 부대시설이 갖춰져 있으며, 직원들은 친절한 서비스로 긍정적인 평가를 받고 있습니다. 조식과 레스토랑 서비스도 만족스럽고, 전반적으로 가성비가 좋다는 후기가 많습니다. 다만, 체크인 과정이 다소 길어 불편이 있었던 점도 언급되었습니다."""
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.

다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot_real}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries_1shot = [summarize(reviews_1shot, prompt_1shot, temperature=1.0) for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews_1shot, summaries_1shot, [summary_1shot_real for _ in range(eval_count)])

print(f'Wins : {wins}, Losses : {losses}, Ties : {ties}')

# filtered_count : 109


100%|██████████| 10/10 [00:41<00:00,  4.10s/it]

Wins : 9, Losses : 1, Ties : 0


In [40]:
summaries_1shot

['숙소는 편안한 침대와 깨끗한 실내, 그리고 매력적인 도심뷰가 인상적이라는 평을 받고 있습니다. 용산역과 아이파크몰과의 연결로 접근성이 뛰어나며, 무료로 이용할 수 있는 수영장과 헬스장 등의 부대시설도 제공되어 호캉스에 적합합니다. 직원들의 친절한 서비스와 조식, 룸 서비스 또한 많은 투숙객들에게 긍정적인 반응을 얻고 있습니다. 하지만 청결 상태와 관련해 아쉬움이 occasionally 언급되며, 체크인 및 체크아웃 과정에서도 다소의 불편함이 있다는 후기도 있습니다. 전반적으로 재방문 의사가 높다는 의견이 많습니다.',
 '숙소는 용산역과 아이파크몰과 바로 연결되어 있어 접근성이 뛰어나며, 주변 관광지로의 이동도 편리합니다. 객실은 깨끗하고 넓으며, 침구는 매우 편안해 피로를 풀기에 적합합니다. 또한, 친절한 직원들이 고객 서비스를 제공하고, 다양한 부대시설도 잘 갖춰져 있습니다. 좋은 도심 뷰가 있으며 조식은 가성비가 뛰어난 편이라고 많은 이용객들이 만족감을 표시하고 있습니다. 다만, 청결 상태에서 아쉬움을 보인 리뷰도 있어 개선의 여지가 있다는 의견도 있었습니다.',
 '서울의 이 숙소는 편안한 침대와 깨끗한 실내 환경이 인상적입니다. 특히 도심뷰가 아름다워 많은 투숙객들이 긍정적인 평가를 남겼습니다. 용산역과 가까워 접근성이 좋으며, 직원들의 친절한 서비스와 다양한 부대시설이 큰 장점으로 부각되고 있습니다. 조식과 해피아워 서비스도 만족스럽다는 후기가 많아, 다양한 혜택이 제공되는 점이 매력적이라고 할 수 있습니다. 다만, 일부 고객들은 청소 상태나 룸타입 변경에 대한 아쉬움을 표현하기도 하였습니다.',
 '숙소는 용산역과 가까워 접근성이 뛰어나며, 주변에 아이파크몰이 있어 편리합니다. 객실은 청결하고 편안한 침대가 특징이며, 도심뷰가 매우 아름답습니다. 친절한 직원들이 다양한 편의시설을 관리하고 있어 서비스도 만족스러운 평가를 받고 있습니다. 조식과 수영장, 헬스장과 같은 부대시설도 호평을 받고 있으며, 전반적으로 재방문 의사가 높습니다. 다만, 일부

In [44]:
summaries = [summarize(reviews, prompt_1shot, temperature=1.0) for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240915 for _ in range(eval_count)])

print(f'Wins : {wins}, Losses : {losses}, Ties : {ties}')

summaries

100%|██████████| 10/10 [00:48<00:00,  4.83s/it]

Wins : 7, Losses : 3, Ties : 0


['글래드 코엑스점은 접근성이 뛰어난 위치에 있으며, 삼성역과 가까워 편리한 이동이 가능합니다. 내부는 깨끗하고 쾌적하며, 친절한 직원들이 좋은 서비스를 제공하는 점이 긍정적으로 평가됩니다. 침구의 편안함과 조식의 알찬 구성도 많은 찬사를 받고 있으며, 독서등과 같은 세심한 배려가 돋보입니다. 여러 번 재방문하고 싶어하는 고객들이 다수 존재하며, 전반적으로 만족스러운 경험을 제공하는 숙소입니다.',
 '글래드 코엑스점은 접근성이 뛰어난 위치에 있어 삼성역과 코엑스몰과의 연결이 매우 편리합니다. 객실은 깨끗하고 넓어 가족 단위로 방문하기 좋은 환경이었습니다. 직원들의 친절한 서비스와 간편한 셀프 체크인 시스템도 큰 장점으로, 조식 또한 만족스러운 아메리칸 스타일로 제공되어 호평을 받고 있습니다. 전반적으로 좋은 숙소라는 평이 많으며, 재방문 의사도 높습니다.',
 '글래드 코엑스점은 삼성역과 가까워 접근성이 뛰어나며, 코엑스와 다양한 주변 부대시설에 손쉽게 이동할 수 있어 매우 편리합니다. 고객들은 친절한 직원 서비스와 깔끔한 객실을 높게 평가하였으며, 침구가 편안하여 쾌적한 휴식을 제공한다고 합니다. 조식은 간편하고 맛있어 만족스러운 식사를 할 수 있었으며, 다채로운 스테이 형태가 옵션으로 제공되어 많은 호응을 얻고 있습니다. 셀프 체크인 시스템이 도입되어 편리한 체크인 경험을 제공하는 점도 긍정적인 요소로 언급되었습니다.',
 '글래드 코엑스는 삼성역과 가까운 접근성 덕분에 많은 고객들에게 편리한 숙소로 평가받고 있습니다. 스태프분들의 친절한 서비스와 함께 깔끔하고 쾌적한 객실이 인상적이며, 조식 또한 맛있고 알찬 구성으로 만족도가 높습니다. 특히 뷰는 다소 아쉬운 점이 있지만, 넓고 편안한 침대와 조용한 환경 덕분에 편안한 휴식을 취할 수 있었다는 후기가 많습니다. 다양한 부대시설과 환경 친화적인 요소 또한 고객들에게 긍정적인 평가를 받고 있으며, 재방문 의사를 밝히는 이들이 많습니다.',
 '글래드 코엑스점은 교통이 편리한 위치에 있으며, 삼성역과 코엑스

In [ ]:
reviews_2shot, _ = preprocess_reviews_v2('./resource/sofitel_reviews.json')
# summary_1shot = summarize(reviews_1shot, prompt, temperature=0.0)
summary_1shot_real = """용산역과 가까워 접근성이 뛰어나며, 근처에 아이파크몰이 위치해 있어 편리합니다. 객실은 넓고 청결하며, 편안한 침대와 훌륭한 도심 뷰가 특징입니다. 다양한 부대시설이 갖춰져 있으며, 직원들은 친절한 서비스로 긍정적인 평가를 받고 있습니다. 조식과 레스토랑 서비스도 만족스럽고, 전반적으로 가성비가 좋다는 후기가 많습니다. 다만, 체크인 과정이 다소 길어 불편이 있었던 점도 언급되었습니다."""
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.

다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot_real}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries_1shot = [summarize(reviews_1shot, prompt_1shot, temperature=1.0) for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews_1shot, summaries_1shot, [summary_1shot_real for _ in range(eval_count)])

print(f'Wins : {wins}, Losses : {losses}, Ties : {ties}')